In [1]:
import glob
import os

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.schema import Document
import os

pinecone_api_key = os.getenv("PINECONE_API_KEY")

loader = PyPDFDirectoryLoader("DIC")
documents = loader.load()

print(len(documents))

NameError: name 'docs' is not defined

In [9]:
print(pinecone_api_key)

None


In [ ]:
len([f for f in os.listdir("./DIC/")])

In [3]:
documents[4]

Document(metadata={'source': 'DIC/FR0007040373_DIC_FR_20230630.pdf', 'page': 0}, page_content="DOCUMENT D’INFORMATIONS CLÉS \nLe présent document contient des informations essentielles sur le produit d’investissement. Il ne s’agit pas d’un document à caractère commercial. \nCes informations vous sont fournies conformément à une obligation légale, afin de vous aider à comprendre en quoi consiste ce p roduit et quels \nrisques, coûts, gains et pertes potentiels y sont associés, et de vous aider à le comparer à d’autres produits.\nEn quoi consiste ce produit ? \nType : Fonds d'Investissement à Vocation Générale (FIVG) \n \nDurée\n : Le Fonds est créé pour une durée de 99 ans. \n \nObjectifs\n :   \nL’objectif du FCP est, au travers d’une gestion discrétionnaire, la recherche d’une \nperformance sur une durée de placement recommandée de 5 ans, en s’exposant aux \nmarchés financiers français ou étrangers actions, obligations et autres titres de créances \npar le biais d’investissements en t

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Initialisation du séparateur de texte avec des paramètres spécifiques pour diviser le texte
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,  # Taille maximale des morceaux de texte
    chunk_overlap=60,  # Chevauchement entre les morceaux pour garder le contexte
    length_function=len,  # Fonction pour calculer la longueur des morceaux
    separators=["\n\n", "\n"]  # Séparateurs utilisés pour diviser le texte en morceaux
)

# Division du document en morceaux (chunks)
chunks = text_splitter.split_documents(documents=documents)

# Affichage du nombre de morceaux créés à partir du document PDF
print(f"{len(chunks)} chunks ont été créés par le splitter à partir du document PDF.")

1721 chunks ont été créés par le splitter à partir du document PDF.


In [5]:
chunks[500]

Document(metadata={'source': 'DIC/KIDPRIIPs_388409_51596_FRA_FRA_20231030.pdf', 'page': 1}, page_content='SCÉNARIOS DE PERFORMANCELes scénarios défavorable, intermédiaire et favorable présentés représentent des exemples utilisant les meilleure et pire performances, ainsi que laperformance moyenne du Compartiment au cours des 10\xa0dernières années. Les marchés pourraient évoluer très diﬀéremment à l’avenir. Le scénario detensions montre ce que vous pourriez obtenir dans des situations de marché extrêmes.\nCe que vous obtiendrez de ce produit dépend des performances futures du marché. L’évolution future du marché est aléatoire et ne peut êtreprédite avec précision.')

In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Charger le modèle d'encodage de texte BAAI/bge-small-en-v1.5 de HuggingFace
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", encode_kwargs={"normalize_embeddings" : True})

/opt/conda/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
from pinecone import Pinecone, ServerlessSpec
from pinecone.exceptions import NotFoundException

# TODO: Inscrire la clé API Pinecone
pinecone = Pinecone(api_key=pinecone_api_key)

try:
    pinecone.describe_index("rag")
except NotFoundException: 
    # Créer un index nommé "rag" de dimension 384
    pinecone.create_index("rag", dimension=1024, spec=ServerlessSpec(cloud="aws", region="us-east-1"))
except Exception:
    print("Une erreur inconnue est survenue !")

PineconeConfigurationError: You haven't specified an Api-Key.

In [ ]:
from langchain_pinecone.vectorstores import PineconeVectorStore

# Initialiser le VectorStore de LlamaIndex avec l'index de Pinecone
pinecone_index = pinecone.Index("rag")
vector_store = PineconeVectorStore(
    index=pinecone_index,
    embedding=embedding
)

In [ ]:
add_result = vector_store.add_documents(chunks)
print(f"{len(add_result)} vecteurs ont été ajoutés dans Pinecone.")

In [ ]:
len(add_result)